In [12]:
import yfinance as yf
import pandas as pd

dataF = yf.download(tickers='BTC',start='2024-01-01', end='2024-04-30', interval='1d')

[*********************100%%**********************]  1 of 1 completed


In [13]:
dataF=dataF[dataF["High"]!=dataF['Low']] #remove cloed market data
dataF.reset_index(inplace=True)
#dataF

In [14]:
def rejection_signal(df): 
    #bullish signal
    if ( df.Open.iloc[-1] < df.Close.iloc[-1] and
       (df.High.iloc[-1] - df.Close.iloc[-1]) < abs(df.Open.iloc[-1]-df.Close.iloc[-1])/10 and
       (df.Open.iloc[-1] - df.Low.iloc[-1]) > abs(df.Open.iloc[-1]-df.Close.iloc[-1])*5):
        return 2
    
    #bearish signal
    elif ( df.Open.iloc[-1] > df.Close.iloc[-1] and
       (df.High.iloc[-1] - df.Open.iloc[-1]) > abs(df.Open.iloc[-1]-df.Close.iloc[-1])*5 and
       (df.Close.iloc[-1] - df.Low.iloc[-1]) < abs(df.Open.iloc[-1]-df.Close.iloc[-1])/10):
        return 1
    
    #nosignal
    else:
        return 0

def engulfing_signal(df):
    # Get the current and previous candles
    previous_candle = df.iloc[-2]
    current_candle = df.iloc[-1]

    # Check for bullish engulfing
    if ( (current_candle['Close'] > previous_candle['Open']) 
        and (current_candle['Open'] < previous_candle['Close'])
        and (previous_candle['Open'] > previous_candle['Close']) ):
        return 2

    # Check for bearish engulfing
    elif ( (current_candle['Open'] > previous_candle['Close']) 
          and (current_candle['Close'] < previous_candle['Open']) 
          and (previous_candle['Close'] > previous_candle['Open']) ):
        return 1

    # Return 0 for any other case
    else:
        return 0
    

signal = [0]*len(dataF)
for i in range(3,len(dataF)):
    df = dataF[i-3:i+1]
    signal[i]= rejection_signal(df)
dataF["rejection_signal"] = signal

signal = [0]*len(dataF)
for i in range(1,len(dataF)):
    df = dataF[i-1:i+1]
    signal[i]= engulfing_signal(df)
dataF["engulfing_signal"] = signal

C:\Users\alada\AppData\Local\Temp\ipykernel_35692\120112563.py:44: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\alada\AppData\Local\Temp\ipykernel_35692\120112563.py:50: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [15]:
up_count = 0
down_count = 0
total_count = 0

for i in range(len(dataF) - 1):
    if dataF.engulfing_signal.iloc[i] == 1:
        total_count += 1
        if dataF.Close.iloc[i+1] > dataF.Open.iloc[i+1]:
            up_count += 1
        elif dataF.Close.iloc[i+1] < dataF.Open.iloc[i+1]:
            down_count += 1

up_percentage = (up_count / total_count) * 100
down_percentage = (down_count / total_count) * 100

In [16]:
print(up_percentage, down_percentage, total_count)

0.0 100.0 3


In [17]:
dataF[dataF["engulfing_signal"]==1]

,Date,Open,High,Low,Close,Adj Close,Volume,rejection_signal,engulfing_signal
8,2024-01-24,92.190002,92.190002,91.806396,91.806396,91.249931,371,0,1
24,2024-02-27,91.610001,91.610001,91.449997,91.487999,90.933464,724,0,1
32,2024-03-15,91.809998,91.809998,91.730202,91.730202,91.174194,436,0,1


In [18]:
import plotly.graph_objects as go

st = 0
dfpl = dataF[st:st+150].copy()
fig = go.Figure(data=[go.Candlestick(x=dfpl.index, open=dfpl['Open'], high=dfpl['High'], low=dfpl['Low'],    close=dfpl['Close'])])

fig.show()

In [19]:
def average_next_n_candles(df, i, N):
    # Check if there are N candles after the current one
    if i + N >= len(df):
        return None

    # Compute the average closing price of the next N candles
    avg_price = df['Close'].iloc[i+1:i+N+1].mean()

    # Compare the average price to the current closing price
    if avg_price < df['Close'].iloc[i]:
        return 1
    elif avg_price > df['Close'].iloc[i]:
        return 2
    else:
        return 0

N=4
signal = [0]*len(dataF)
for i in range(len(dataF)-N):
    signal[i]= average_next_n_candles(dataF, i, N)
dataF["price_target"] = signal

C:\Users\alada\AppData\Local\Temp\ipykernel_35692\3275755919.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [20]:
dataF[dataF["engulfing_signal"]==dataF["price_target"]].count()

Date                4
Open                4
High                4
Low                 4
Close               4
Adj Close           4
Volume              4
rejection_signal    4
engulfing_signal    4
price_target        4
dtype: int64

In [21]:
equal_count = 0
different_count = 0
total_count = 0

for i in range(len(dataF)):
    if dataF.engulfing_signal.iloc[i] != 0:
        total_count += 1
        if dataF.engulfing_signal.iloc[i] == dataF.price_target.iloc[i]:
            equal_count += 1
        else:
            different_count += 1

equal_percentage = (equal_count / total_count) * 100
different_percentage = (different_count / total_count) * 100

In [22]:
print(equal_percentage, different_percentage)

0.0 100.0
